# HEllloooo

Install wbdata, only needs to be run once even if u leave the notebook

In [ ]:
import sys
!{sys.executable} -m pip install wbdata

In [1]:
# Suppresses warning errors, there are too many of them there about caches from wbdata
import logging
logging.getLogger("shelved_cache.persistent_cache").setLevel(logging.ERROR)

Listing the names of the datasets

In [2]:
import wbdata
wbdata.get_sources()

  id  name
----  --------------------------------------------------------------------
   1  Doing Business
   2  World Development Indicators
   3  Worldwide Governance Indicators
   5  Subnational Malnutrition Database
   6  International Debt Statistics
  11  Africa Development Indicators
  12  Education Statistics
  13  Enterprise Surveys
  14  Gender Statistics
  15  Global Economic Monitor
  16  Health Nutrition and Population Statistics
  18  IDA Results Measurement System
  19  Millennium Development Goals
  20  Quarterly Public Sector Debt
  22  Quarterly External Debt Statistics SDDS
  23  Quarterly External Debt Statistics GDDS
  25  Jobs
  27  Global Economic Prospects
  28  Global Findex database
  29  The Atlas of Social Protection: Indicators of Resilience and Equity
  30  Exporter Dynamics Database – Indicators at Country-Year Level
  31  Country Policy and Institutional Assessment
  32  Global Financial Development
  33  G20 Financial Inclusion Indicators
  34  Global P

Datasets i thought would be the most valuable for us (not considering the completeness)


In [3]:
print("\n --- Gender Disaggregated Labor Database (GDLD), examples: --- ")
[print(item['name']) for item in wbdata.get_indicators(source=80)[:5]]
print("\n --- Country Climate and Development Report (CCDR), examples: --- ")
[print(item['name']) for item in wbdata.get_indicators(source=87)[:5]]
print("\n  --- Sustainable Development Goals, examples: --- ")
[print(item['name']) for item in wbdata.get_indicators(source=46)[:5]]
print("\n  --- Sustainable Energy for All, examples: --- ")
[print(item['name']) for item in wbdata.get_indicators(source=35)[:5]]
print("\n  --- Gender Statistics, examples: --- ")
[print(item['name']) for item in wbdata.get_indicators(source=14)[:5]]



 --- Gender Disaggregated Labor Database (GDLD), examples: --- 
Value-added from Capital in GTAP10P3COD
Value-added from labor in GTAP10P3COD (in USD 2014 million)
Share of female unskilled worker out of all workers in this sector, %
Share of female skilled worker out of all workers in this sector, %
Share of male unskilled worker out of all workers in this sector, %

 --- Country Climate and Development Report (CCDR), examples: --- 
Account (% age 15+)
Account, income, poorest 40% (% ages 15+)
Agricultural land (% of land area)
Arable land (% of land area)
Forest area (% of land area)

  --- Sustainable Development Goals, examples: --- 
Forest area (sq. km)
Forest area (% of land area)
Cereal yield (kg per hectare)
Current account balance (% of GDP)
Portfolio Investment, net (BoP, current US$)

  --- Sustainable Energy for All, examples: --- 
Access to electricity (% of total population)
Total final energy consumption (TFEC)
Access to electricity (% of rural population)
Access to ele

[None, None, None, None, None]

### Checking completeness of the datasets, best indicators

In [26]:
import wbdata
import pandas as pd
from datetime import datetime
from tqdm import tqdm

def rank_indicators_by_completeness(source=80, start_year=2000, end_year=2020, top_n=10):
    """
    Fetch all indicators from a source and rank them by data completeness.
    Fetches data in a single call by querying all countries at once.
    """
    # Get all indicators for the source
    indicators = wbdata.get_indicators(source=source)
    indicator_dict = {ind['id']: ind['name'] for ind in indicators}
    
    print(f"Found {len(indicator_dict)} indicators in source {source}")
    print("Fetching data for all indicators...")
    
    # Fetch data using the API more efficiently - get all data at once
    try:
        date_range = (datetime(start_year, 1, 1), datetime(end_year, 1, 1))
        
        # Get data for all indicators by passing them as a list to fetch parameter
        all_data = []
        for code in tqdm(indicator_dict.keys(), desc="Fetching indicators"):
            try:
                data = wbdata.get_data(
                    code,
                    date=date_range,
                    freq='Y'
                )
                all_data.extend([(code, d['country']['value'], d['date'], d['value']) 
                                for d in data if d['value'] is not None])
            except:
                continue
        
        # Convert to dataframe
        df = pd.DataFrame(all_data, columns=['indicator', 'country', 'date', 'value'])
        
        print(f"Retrieved {len(df)} data points")
        
        # Calculate completeness for each indicator
        results = []
        total_possible = len(df['country'].unique()) * (end_year - start_year + 1)
        
        for code, name in tqdm(indicator_dict.items(), desc="Calculating completeness"):
            indicator_data = df[df['indicator'] == code]
            count = len(indicator_data)
            completeness = count / total_possible if total_possible > 0 else 0
            results.append((code, name, completeness, count))
        
        # Create and sort results
        summary = pd.DataFrame(results, columns=["Indicator Code", "Indicator Name", "Completeness", "Data Points"])
        top = summary.sort_values(by="Completeness", ascending=False)   
        
        return top, df
        
    except Exception as e:
        print(f"Error fetching data: {e}")
        import traceback
        traceback.print_exc()
        return None, None

### Gender Disaggregated Labor Database (GDLD) doesnt work for me :(

In [15]:
gdld_summary, gdld_df = rank_indicators_by_completeness(80, 2000, 2025, 10)

Found 16 indicators in source 80
Fetching data for all indicators...


Fetching indicators: 100%|█████████████████████████████████████████████████████████████| 16/16 [00:02<00:00,  6.69it/s]


Retrieved 0 data points


Calculating completeness: 100%|██████████████████████████████████████████████████████| 16/16 [00:00<00:00, 2370.00it/s]


In [50]:
print(gdld_summary)

    Indicator Code                                     Indicator Name  \
0  gtap10VACapital            Value-added from Capital in GTAP10P3COD   
1    gtap10VALabor  Value-added from labor in GTAP10P3COD (in USD ...   
2          p_F_nsk  Share of female unskilled worker out of all wo...   
3          p_F_skl  Share of female skilled worker out of all work...   
4          p_M_nsk  Share of male unskilled worker out of all work...   
5          p_M_skl  Share of male skilled worker out of all worker...   
6          v_F_nsk       Labor volume for unskilled females, millions   
7          v_F_skl         Labor volume for skilled females, millions   
8          v_M_nsk          Labor volume for unskilled male, millions   
9          v_M_skl           Labor volume for skilled males, millions   

   Completeness  Data Points  
0             0            0  
1             0            0  
2             0            0  
3             0            0  
4             0            0  
5         

Sustainable Development Goals indicators completeness:

In [17]:
sdg_summary, sdg_df = rank_indicators_by_completeness(46, 2000, 2025, 10)

Found 408 indicators in source 46
Fetching data for all indicators...


Fetching indicators: 100%|███████████████████████████████████████████████████████████| 408/408 [13:43<00:00,  2.02s/it]


Retrieved 1234246 data points


Calculating completeness: 100%|██████████████████████████████████████████████████████| 408/408 [00:27<00:00, 14.82it/s]


In [35]:
print(sdg_summary.head(15))

        Indicator Code                                     Indicator Name  \
396  SP.URB.TOTL.IN.ZS           Urban population (% of total population)   
395        SP.URB.TOTL                                   Urban population   
394        SP.URB.GROW                 Urban population growth (annual %)   
111     NY.GDP.MKTP.CD                                  GDP (current US$)   
119     NY.GDP.PCAP.CD                       GDP per capita (current US$)   
385        SP.ADO.TFRT  Adolescent fertility rate (births per 1,000 wo...   
114  NY.GDP.MKTP.KD.ZG                              GDP growth (annual %)   
122  NY.GDP.PCAP.KD.ZG                   GDP per capita growth (annual %)   
121     NY.GDP.PCAP.KD                 GDP per capita (constant 2015 US$)   
113     NY.GDP.MKTP.KD                            GDP (constant 2015 US$)   

     Completeness  Data Points  
396      0.829130         6575  
395      0.829130         6575  
394      0.829004         6574  
111      0.808701   

## Country Climate and Development Report

In [27]:
ccdr_summary, ccdr_df = rank_indicators_by_completeness(87, 2000, 2025, 10)

Found 432 indicators in source 87
Fetching data for all indicators...


Fetching indicators: 100%|█████████████████████████████████████████████████████████| 432/432 [1:10:11<00:00,  9.75s/it]


Retrieved 815635 data points


Calculating completeness: 100%|██████████████████████████████████████████████████████| 432/432 [00:17<00:00, 25.00it/s]


In [28]:
print(ccdr_summary.head(15))

        Indicator Code                                     Indicator Name  \
429        SP.POP.TOTL                                  Population, total   
430  SP.URB.TOTL.IN.ZS           Urban population (% of total population)   
411     NY.GDP.PCAP.CD                       GDP per capita (current US$)   
408     NY.GDP.MKTP.CD                                  GDP (current US$)   
409  NY.GDP.MKTP.KD.ZG                              GDP growth (annual %)   
394     EG.ELC.ACCS.ZS            Access to electricity (% of population)   
4       AG.LND.FRST.ZS                       Forest area (% of land area)   
3       AG.LND.ARBL.ZS                       Arable land (% of land area)   
423     SH.STA.BASS.ZS  People using at least basic sanitation service...   
422     SH.H2O.BASW.ZS  People using at least basic drinking water ser...   

     Completeness  Data Points  
429      0.926573         6625  
430      0.919580         6575  
411      0.896923         6413  
408      0.896923   

## Sustainable Energy for All indicators completeness:

In [29]:
se_summary, se_df = rank_indicators_by_completeness(35, 2000, 2025, 10)

Found 11 indicators in source 35
Fetching data for all indicators...


Fetching indicators: 100%|█████████████████████████████████████████████████████████████| 11/11 [00:10<00:00,  1.07it/s]


Retrieved 39126 data points


Calculating completeness: 100%|███████████████████████████████████████████████████████| 11/11 [00:00<00:00, 471.53it/s]


In [49]:
print(se_summary.head(15))

                   Indicator Code  \
1   1.1_TOTAL.FINAL.ENERGY.CONSUM   
6              3.1_RE.CONSUMPTION   
8    4.1.2_REN.ELECTRICITY.OUTPUT   
9     4.1_SHARE.RE.IN.ELECTRICITY   
0      1.1_ACCESS.ELECTRICITY.TOT   
3    1.3_ACCESS.ELECTRICITY.URBAN   
7  4.1.1_TOTAL.ELECTRICITY.OUTPUT   
5      2.1_SHARE.TOTAL.RE.IN.TFEC   
2    1.2_ACCESS.ELECTRICITY.RURAL   
4              2.1_ACCESS.CFT.TOT   

                                      Indicator Name  Completeness  \
1              Total final energy consumption (TFEC)      0.610782   
6                  Renewable energy consumption (TJ)      0.610782   
8          Renewable energy electricity output (GWh)      0.610618   
9  Renewable electricity (% in total electricity ...      0.610618   
0      Access to electricity (% of total population)      0.604043   
3      Access to electricity (% of urban population)      0.604043   
7                     Total electricity output (GWh)      0.599441   
5            Renewable energy co

## Gender Statistics indicators completeness:

In [31]:
gs_summary, gs_df = rank_indicators_by_completeness(14, 2000, 2025, 10)

Found 1277 indicators in source 14
Fetching data for all indicators...


Fetching indicators: 100%|██████████████████████████████████████████████████████| 1277/1277 [13:06:22<00:00, 36.95s/it]


Retrieved 480728 data points


Calculating completeness: 100%|████████████████████████████████████████████████████| 1277/1277 [00:30<00:00, 41.73it/s]


In [32]:
print(gs_summary.head(15))

        Indicator Code                                    Indicator Name  \
439     NY.GDP.PCAP.CD                      GDP per capita (current US$)   
437     NY.GDP.MKTP.CD                                 GDP (current US$)   
438  NY.GDP.MKTP.KD.ZG                             GDP growth (annual %)   
440     NY.GDP.PCAP.KD                GDP per capita (constant 2015 US$)   
442     NY.GNP.PCAP.CD        GNI per capita, Atlas method (current US$)   
441     NY.GNP.ATLS.CD                   GNI, Atlas method (current US$)   
443  NY.GNP.PCAP.PP.CD     GNI per capita, PPP (current international $)   
430     IT.NET.USER.ZS  Individuals using the Internet (% of population)   
479        SE.PRM.ENRR              School enrollment, primary (% gross)   
480     SE.PRM.ENRR.FE      School enrollment, primary, female (% gross)   

     Completeness  Data Points  
439      0.862426         6413  
437      0.862426         6413  
438      0.852743         6341  
440      0.849785         6319 